# Tutorial Q2 - Variational quantum eigensolver

This tutorial demonstrates a simplified variational quantum eigensolver (Peruzzo et al 2014). To showcase the hybrid principle of openqml, we first train a quantum circuit to minimize the energy expectation for a Hamiltonian $H$, 

$$ \langle \psi | H | \psi \rangle  = 0.1 \langle \psi_{v} | X_2 | \psi_v \rangle + 0.5 \langle \psi_v | Y_2 | \psi_v \rangle.  $$

Here, $| \psi_v \rangle $ is the state after applying the quantum circuit which depends on trainable variables $v = \{v_1, v_2\}$, and $X_2$, $Y_2$ denote the Pauli-X and Pauli-Y operator acting on the second qubit. 

We then turn things around and use a fixed quantum circuit to prepare $| \psi \rangle $, but train the coefficients of the Hamiltonian to minimize

$$ \langle \psi | H | \psi \rangle  = v_1 \langle \psi | X_2 | \psi \rangle + v_2 \langle \psi | Y_2 | \psi \rangle . $$


## 1. Optimize the quantum circuit

### Imports

Alongside the openqml framework and gradient descent optimizer, we import the original numpy library. 

*Note: If we perform numpy operations in the cost function or any function the cost depends on, we have to use the openqml version of numpy imported as `from openqml import numpy as onp`.*

In [1]:
import openqml as qm
from openqml.optimize import GradientDescentOptimizer
import numpy as np

We use the projectq simulator as a device.

In [2]:
dev = qm.device('projectq.simulator', wires=2)

### Quantum nodes

The quantum circuit of the variational eigensolver is an ansatz that defines a manifold of possible quantum states. We use a Hadamard, two rotations and a CNOT gate. 

In [3]:
def ansatz(vars):

    qm.Hadamard([0])
    qm.RX(vars[0], [0])
    qm.RY(vars[1], [1])
    qm.CNOT([0, 1])

A variational eigensolvers requires us to evaluate expectations of different Pauli operators. In this example, the Hamiltonian is expressed by only two single-qubit Pauli operators, namely the X and Y operator applied to the first qubit. 

Since these operators do not commute, we need two quantum nodes, but they can reuse the same device we created. 

*NOTE: If the Pauli observables referred to different qubits, we could use one quantum function and return a tuple of expectations in only one quantum node: `return qm.expectation.PauliX(0), qm.expectation.PauliX(1)`*

In [4]:
@qm.qnode(dev)
def circuit_X(vars):
    ansatz(vars)
    return qm.expval.PauliX(1)


@qm.qnode(dev)
def circuit_Y(vars):
    ansatz(vars)
    return qm.expval.PauliY(1)

### Objective

The objective of a VQE, usually called a "cost", is simply a linear combination of the expectations, which defines the expectation of the Hamiltonian we are interested in. 

In [5]:
def cost(vars):

    expX = circuit_X(vars)
    expY = circuit_Y(vars)

    return 0.1*expX + 0.5*expY

This cost defines the following landscape:

*Note: To run the following cell you need the matplotlib library.*

In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import MaxNLocator

fig = plt.figure(figsize = (6, 4))
ax = fig.gca(projection='3d')

X = np.linspace(-3.1, 3.1, 30)
Y = np.linspace(-3.1, 3.1, 30)
xx, yy = np.meshgrid(X, Y)
Z = np.array([[cost([x, y]) for x in X] for y in Y]).reshape(len(Y), len(X))
surf = ax.plot_surface(xx, yy, Z, cmap=cm.coolwarm, antialiased=False)

ax.set_xlabel("v1")
ax.set_ylabel("v2")
ax.zaxis.set_major_locator(MaxNLocator(nbins = 5, prune = 'lower'))

plt.show()

<Figure size 600x400 with 1 Axes>

### Optimization

We use the gradient descent optimizer from Tutorial 1.

In [7]:
o = GradientDescentOptimizer(0.5)

vars = np.array([0., 0.])
vars_gd = [vars]
for it in range(20):
    vars = o.step(cost, vars)
    vars_gd.append(vars)

    print('Cost after step {:5d}: {: .7} | Variables: [{: .5},{: .5}]'
          .format(it+1, cost(vars), vars[0], vars[1]) )

Cost after step     1: -0.004997917 | Variables: [ 0.0,-0.05]
Cost after step     2: -0.009977124 | Variables: [ 0.0,-0.099938]
Cost after step     3: -0.01491297 | Variables: [-3.4694e-19,-0.14969]
Cost after step     4: -0.01978155 | Variables: [-3.4694e-19,-0.19913]
Cost after step     5: -0.02456022 | Variables: [-3.4694e-19,-0.24814]
Cost after step     6: -0.02922794 | Variables: [-3.4694e-19,-0.29661]
Cost after step     7: -0.03376565 | Variables: [-3.4694e-19,-0.34443]
Cost after step     8: -0.03815657 | Variables: [-1.7347e-18,-0.39149]
Cost after step     9: -0.04238634 | Variables: [-1.7347e-18,-0.43771]
Cost after step    10: -0.04644318 | Variables: [-3.1225e-18,-0.48299]
Cost after step    11: -0.05031789 | Variables: [-3.1225e-18,-0.52727]
Cost after step    12: -0.05400382 | Variables: [-3.1225e-18,-0.57048]
Cost after step    13: -0.05749676 | Variables: [-3.1225e-18,-0.61256]
Cost after step    14: -0.06079478 | Variables: [-3.1225e-18,-0.65347]
Cost after step    1

We can plot the path gradient descent took.

In [ ]:
fig = plt.figure(figsize = (6, 4))
ax = fig.gca(projection='3d')

X = np.linspace(-3, 3, 20)
Y = np.linspace(-3, 3, 20)
xx, yy = np.meshgrid(X, Y)
Z = np.array([[cost([x, y]) for x in X] for y in Y]).reshape(len(Y), len(X))
surf = ax.plot_surface(xx, yy, Z, cmap=cm.coolwarm, antialiased=False)

path_z = [cost(vars)+1e-8 for vars in vars_gd]
path_x = [v[0] for v in vars_gd]
path_y = [v[1] for v in vars_gd]
ax.plot(path_x, path_y, path_z, c='green', marker='.', label="graddesc")

ax.set_xlabel("v1")
ax.set_ylabel("v2")
ax.zaxis.set_major_locator(MaxNLocator(nbins = 5, prune = 'lower'))

plt.legend()
plt.show()

## 2. Optimizing the Hamiltonian coefficients

Instead of optimizing the circuit parameters, we can also use a fixed circuit,

In [9]:
def ansatz():

    qm.Hadamard([0])
    qm.RX(-0.5, [0])
    qm.RY( 0.5, [1])
    qm.CNOT([0, 1])
    
    
@qm.qnode(dev)
def circuit_X():
    """Circuit measuring the X operator for the second qubit"""
    ansatz()
    return qm.expval.PauliX(1)


@qm.qnode(dev)
def circuit_Y():
    """Circuit measuring the Y operator for the second qubit"""
    ansatz()
    return qm.expval.PauliY(1)

and make the classical coefficients trainable.

In [10]:
def cost(vars):

    expX = circuit_X()
    expY = circuit_Y()

    return vars[0]*expX + vars[1]*expY

With this cost, ever smaller coefficients decrease the energy expectation.

In [11]:
o = GradientDescentOptimizer(0.5)

vars = np.array([0., 0.])
vars_gd = [vars]
for it in range(20):
    vars = o.step(cost, vars)
    vars_gd.append(vars)


    print('Cost after step {:5d}: {: .7} | Variables: [{: .5},{: .5}]'
          .format(it+1, cost(vars), vars[0], vars[1]) )

Cost after step     1: -0.1149244 | Variables: [-0.23971, 0.0]
Cost after step     2: -0.2298488 | Variables: [-0.47943, 0.0]
Cost after step     3: -0.3447733 | Variables: [-0.71914, 0.0]
Cost after step     4: -0.4596977 | Variables: [-0.95885, 0.0]
Cost after step     5: -0.5746221 | Variables: [-1.1986, 0.0]
Cost after step     6: -0.6895465 | Variables: [-1.4383, 0.0]
Cost after step     7: -0.804471 | Variables: [-1.678, 0.0]
Cost after step     8: -0.9193954 | Variables: [-1.9177, 0.0]
Cost after step     9: -1.03432 | Variables: [-2.1574, 0.0]
Cost after step    10: -1.149244 | Variables: [-2.3971, 0.0]
Cost after step    11: -1.264169 | Variables: [-2.6368, 0.0]
Cost after step    12: -1.379093 | Variables: [-2.8766, 0.0]
Cost after step    13: -1.494018 | Variables: [-3.1163, 0.0]
Cost after step    14: -1.608942 | Variables: [-3.356, 0.0]
Cost after step    15: -1.723866 | Variables: [-3.5957, 0.0]
Cost after step    16: -1.838791 | Variables: [-3.8354, 0.0]
Cost after step 

Of course, the optimization landscape is nearly linear.

In [ ]:
fig = plt.figure(figsize = (6, 4))
ax = fig.gca(projection='3d')

X = np.linspace(-3, 3, 50)
Y = np.linspace(-3, 3, 50)
xx, yy = np.meshgrid(X, Y)
Z = np.array([[cost([x, y]) for x in X] for y in Y]).reshape(len(Y), len(X))
surf = ax.plot_surface(xx, yy, Z, cmap=cm.coolwarm, antialiased=False)

path_z = [cost(vars)+1e-8 for vars in vars_gd]
path_x = [v[0] for v in vars_gd]
path_y = [v[1] for v in vars_gd]
ax.plot(path_x, path_y, path_z, c='green', marker='.', label="graddesc")

ax.set_xlabel("v1")
ax.set_ylabel("v2")
ax.zaxis.set_major_locator(MaxNLocator(nbins = 5, prune = 'lower'))

plt.legend()
plt.show()

 <img src="figures/vqe_c_landscape_gd.png" width="450"> 

## 3. Optimizing classical and quantum parameters

Of course, we can also optimize "classical" and "quantum" weights together by combining the two approaches from above.

In [ ]:
def ansatz(vars):

    qm.Hadamard([0])
    qm.RX(vars[0], [0])
    qm.RY(vars[1], [1])
    qm.CNOT([0, 1])


@qm.qnode(dev)
def circuit_X(vars):
    ansatz(vars)
    return qm.expval.PauliX(1)


@qm.qnode(dev)
def circuit_Y(vars):
    ansatz(vars)
    return qm.expval.PauliY(1)


def cost(vars):

    expX = circuit_X(vars)
    expY = circuit_Y(vars)

    return vars[2]*expX + vars[3]*expY

weights0 = np.array([0., 0., 0., 0.])
print('Initial weights:', weights0)

o = GradientDescentOptimizer(0.5)
weights = weights0
for iteration in np.arange(1, 21):
    weights = o.step(cost, weights)
    print('Cost after step {:5d}: {: 0.7f}'.format(iteration, cost(weights)))
print('Optimized weights:', weights)